In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

print("Google Drive mounted successfully!")

Mounted at /content/drive/
Google Drive mounted successfully!


In [ ]:
import pandas as pd
import joblib
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

# Set up the device (use GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# --- Define Path and Load Data ---

# Define the ONE TRUE PATH to your project folder in Google Drive.
# All other cells will use this variable.
drive_path = '/content/drive/MyDrive/LUNG-RP-XAI/Thesis_Project/'

# Load our pre-split data using the correct path variable
data = joblib.load(drive_path + 'Results/split_data.joblib')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Path defined and data loaded/scaled successfully.")

Path defined and data loaded/scaled successfully.


In [ ]:
# Convert our numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled.astype(np.float32)).to(device)
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).to(device)
X_test_tensor = torch.tensor(X_test_scaled.astype(np.float32)).to(device)
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).to(device)

# Reshape y tensors to be [n_samples, 1]
y_train_tensor = y_train_tensor.view(y_train_tensor.shape[0], 1)
y_test_tensor = y_test_tensor.view(y_test_tensor.shape[0], 1)

print("Data converted to PyTorch Tensors.")
print(f"Shape of X_train_tensor: {X_train_tensor.shape}")

Data converted to PyTorch Tensors.
Shape of X_train_tensor: torch.Size([336, 107])


In [ ]:
class RadiomicsDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataset objects
train_dataset = RadiomicsDataset(X_train_tensor, y_train_tensor)
test_dataset = RadiomicsDataset(X_test_tensor, y_test_tensor)

print("Custom PyTorch Datasets created.")

Custom PyTorch Datasets created.


In [ ]:
# Define batch size
BATCH_SIZE = 32

# Create DataLoader objects
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("DataLoaders created. Ready for training.")

DataLoaders created. Ready for training.


In [ ]:
# --- Define the Model Architecture ---

# Get the number of input features from our training data
input_features = X_train_scaled.shape[1]

# Define the model using nn.Sequential for a simple feedforward network
mlp_model = nn.Sequential(
    nn.Linear(input_features, 64), # Input layer -> Hidden layer 1
    nn.ReLU(),                     # Activation function for Hidden layer 1
    nn.Dropout(0.3),               # Dropout layer to prevent overfitting

    nn.Linear(64, 32),             # Hidden layer 1 -> Hidden layer 2
    nn.ReLU(),                     # Activation function for Hidden layer 2
    nn.Dropout(0.3),               # Dropout layer

    nn.Linear(32, 1),              # Hidden layer 2 -> Output layer
    nn.Sigmoid()                   # Sigmoid activation to output a probability (0 to 1)
).to(device)


# Print the model architecture
print("--- Model Architecture ---")
print(mlp_model)

--- Model Architecture ---
Sequential(
  (0): Linear(in_features=107, out_features=64, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=64, out_features=32, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=32, out_features=1, bias=True)
  (7): Sigmoid()
)


In [ ]:
# Define the loss function
# BCEWithLogitsLoss is a good choice for binary classification as it's numerically stable
loss_function = nn.BCEWithLogitsLoss()

# Define the optimizer
# Adam is a popular and effective optimization algorithm
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.001)

print("Loss function and optimizer are defined.")

Loss function and optimizer are defined.


In [ ]:
# --- The Training Loop ---
NUM_EPOCHS = 50
best_test_loss = float('inf')

for epoch in range(NUM_EPOCHS):
    # --- Training ---
    mlp_model.train() # Set the model to training mode
    train_loss = 0.0
    for features, labels in train_loader:
        # Move data to the device
        features, labels = features.to(device), labels.to(device)

        # Forward pass: get predictions
        outputs = mlp_model(features)
        # Calculate loss
        loss = loss_function(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad() # Clear previous gradients
        loss.backward()       # Calculate new gradients
        optimizer.step()      # Update model weights

        train_loss += loss.item()

    # --- Validation ---
    mlp_model.eval() # Set the model to evaluation mode
    test_loss = 0.0
    with torch.no_grad(): # We don't need to calculate gradients during evaluation
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = mlp_model(features)
            loss = loss_function(outputs, labels)
            test_loss += loss.item()

    # Print statistics for the epoch
    avg_train_loss = train_loss / len(train_loader)
    avg_test_loss = test_loss / len(test_loader)
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}')

    # Save the model if it has the best test loss so far
    if avg_test_loss < best_test_loss:
        best_test_loss = avg_test_loss
        torch.save(mlp_model.state_dict(), drive_path + 'Results/best_mlp_model.pth')
        print(f'   -> New best model saved with test loss: {best_test_loss:.4f}')

print("\n--- Training Complete ---")

Epoch [1/50], Train Loss: 0.7675, Test Loss: 0.7586
   -> New best model saved with test loss: 0.7586
Epoch [2/50], Train Loss: 0.7445, Test Loss: 0.7410
   -> New best model saved with test loss: 0.7410
Epoch [3/50], Train Loss: 0.7261, Test Loss: 0.7282
   -> New best model saved with test loss: 0.7282
Epoch [4/50], Train Loss: 0.7088, Test Loss: 0.7177
   -> New best model saved with test loss: 0.7177
Epoch [5/50], Train Loss: 0.7000, Test Loss: 0.7099
   -> New best model saved with test loss: 0.7099
Epoch [6/50], Train Loss: 0.6984, Test Loss: 0.7058
   -> New best model saved with test loss: 0.7058
Epoch [7/50], Train Loss: 0.6929, Test Loss: 0.7021
   -> New best model saved with test loss: 0.7021
Epoch [8/50], Train Loss: 0.6942, Test Loss: 0.7001
   -> New best model saved with test loss: 0.7001
Epoch [9/50], Train Loss: 0.6911, Test Loss: 0.6988
   -> New best model saved with test loss: 0.6988
Epoch [10/50], Train Loss: 0.6917, Test Loss: 0.6986
   -> New best model saved wi

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
import matplotlib.pyplot as plt

# --- Evaluation ---

# First, create a new instance of the model architecture
final_model = nn.Sequential(
    nn.Linear(X_train_scaled.shape[1], 64),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(32, 1),
    nn.Sigmoid()
).to(device)

# Load the weights of the best model we saved during training
model_path = drive_path + 'Results/best_mlp_model.pth'
final_model.load_state_dict(torch.load(model_path))
final_model.eval() # Set the model to evaluation mode

# Make predictions on the test set
with torch.no_grad():
    y_pred_proba_tensor = final_model(X_test_tensor)

# Convert predictions and true labels from GPU tensors to CPU numpy arrays
y_pred_proba = y_pred_proba_tensor.cpu().numpy().flatten()
y_pred = (y_pred_proba > 0.5).astype(int)
y_test_cpu = y_test_tensor.cpu().numpy().flatten()


# --- Print Reports ---
print("--- Classification Report (PyTorch MLP Model) ---")
print(classification_report(y_test_cpu, y_pred))

print("\n--- Confusion Matrix (PyTorch MLP Model) ---")
cm = confusion_matrix(y_test_cpu, y_pred)
print("                Predicted: 0 | Predicted: 1")
print(f"Actual: 0      {cm[0,0]:<13} | {cm[0,1]:<13}")
print(f"Actual: 1      {cm[1,0]:<13} | {cm[1,1]:<13}")

roc_auc_mlp = roc_auc_score(y_test_cpu, y_pred_proba)
print(f"\nXGBoost ROC AUC Score: 0.5700")
print(f"PyTorch MLP ROC AUC Score: {roc_auc_mlp:.4f}")

--- Classification Report (PyTorch MLP Model) ---
              precision    recall  f1-score   support

         0.0       0.65      0.96      0.77        50
         1.0       0.80      0.24      0.36        34

    accuracy                           0.67        84
   macro avg       0.72      0.60      0.57        84
weighted avg       0.71      0.67      0.61        84


--- Confusion Matrix (PyTorch MLP Model) ---
                Predicted: 0 | Predicted: 1
Actual: 0      48            | 2            
Actual: 1      26            | 8            

XGBoost ROC AUC Score: 0.5700
PyTorch MLP ROC AUC Score: 0.5153
